In [1]:
from gurobipy import GRB
import gurobipy as gb


# QUESTION 1

## Extracting the necessary data

In [2]:
import pandas as pd

# Load data files
file_paths = {
    "direct_capacity": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Capacity_for_Direct_Production_Facilities.csv",
    "transship_capacity_centers": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Capacity_for_Transship_Distribution_Centers.csv",
    "transship_capacity_facilities": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Capacity_for_Transship_Production_Facilities.csv",
    "cost_production_refinement": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Cost_Production_to_Refinement.csv",
    "cost_production_transshipment": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Cost_Production_to_Transshipment.csv",
    "cost_transshipment_refinement": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Cost_Transshipment_to_Refinement.csv",
    "refinement_demand": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Refinement_Demand.csv",
}

# Reading the data
data = {key: pd.read_csv(file_paths[key]) for key in file_paths}

# Displaying the keys to understand the structure of the data
data.keys()  


dict_keys(['direct_capacity', 'transship_capacity_centers', 'transship_capacity_facilities', 'cost_production_refinement', 'cost_production_transshipment', 'cost_transshipment_refinement', 'refinement_demand'])

In [3]:
# Displaying the first few rows of each dataset to understand their structure
data_overview = {key: data[key].head() for key in data}

data_overview


{'direct_capacity':    ProductionFacility  Capacity
 0                   1       462
 1                   2       103
 2                   3       460
 3                   4       325
 4                   5       227,
 'transship_capacity_centers':    TransshipmentHub  Capacity
 0                 1      1317
 1                 2      1453,
 'transship_capacity_facilities':    ProductionFacility  Capacity
 0                   1       374
 1                   2       444
 2                   3       395
 3                   4       245
 4                   5       378,
 'cost_production_refinement':    ProductionFacility  RefinementCenter      Cost
 0                   1                 1  4.252733
 1                   1                 2  4.567726
 2                   1                 3  4.696484
 3                   1                 4  2.678741
 4                   1                 5  4.272451,
 'cost_production_transshipment':    ProductionFacility  TransshipmentHub      Cost
 0   

In [4]:
data['refinement_demand']

,RefinementCenter,Demand
0,1,1537
1,2,1748
2,3,1940
3,4,1838
4,5,1665


In [5]:
data['cost_production_refinement']

,ProductionFacility,RefinementCenter,Cost
0,1,1,4.252733
1,1,2,4.567726
2,1,3,4.696484
3,1,4,2.678741
4,1,5,4.272451
...,...,...,...
120,25,1,4.384176
121,25,2,5.943448
122,25,3,4.999220
123,25,4,4.154833


In [6]:
# Extracting the necessary data

# Capacities array
cap_direct = data['direct_capacity']['Capacity'].values  
cap_transship_facilities = data['transship_capacity_facilities']['Capacity'].values
cap_transship_centers = data['transship_capacity_centers']['Capacity'].values

# Costs dataframe
costs_prod_ref = data['cost_production_refinement']
costs_prod_trans = data['cost_production_transshipment']
costs_trans_ref = data['cost_transshipment_refinement']



# Number of facilities and centers
num_direct_facilities = len(cap_direct)
num_transship_facilities = len(cap_transship_facilities)
num_refinement_centers = 5  # Given in the problem statement
num_transship_centers = len(cap_transship_centers)

# demand 
demand_data = data["refinement_demand"]

#total demand
total_demand = demand_data['Demand'].sum()

print('Capacity of direct facilities:', cap_direct)
print('Capacity of transship facilities:', cap_transship_facilities)
print('Capacity of distribution center:',cap_transship_centers)


print('\n')


print('number of direct facility:', num_direct_facilities)
print('number of transship facility:', num_transship_facilities)
print('number of distirbution center', num_transship_centers)
print('number of refinement center:',num_refinement_centers)

print('\n')

print('Demand:', demand_data)

print('\n')

print('total_demand:',total_demand)



Capacity of direct facilities: [462 103 460 325 227 217 205 521 548 191 361 411 104 155 285 109 422 438
 501 139 462 504 106 132 298]
Capacity of transship facilities: [374 444 395 245 378 408 435 175 415 503 184 297 450 169 365]
Capacity of distribution center: [1317 1453]


number of direct facility: 25
number of transship facility: 15
number of distirbution center 2
number of refinement center: 5


Demand:    RefinementCenter  Demand
0                 1    1537
1                 2    1748
2                 3    1940
3                 4    1838
4                 5    1665


total_demand: 8728


## Create the optimization model

In [7]:
from gurobipy import GRB, Model

# Create the optimization model
model_a = Model("Transshipment Problem")


Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


## Define decision variables

In [8]:
# Create decision variables
# Xij - Quantity from direct production facility i to refinement center j
X = model_a.addVars(num_direct_facilities, num_refinement_centers,lb =0, vtype=GRB.CONTINUOUS, name="X")

# Yij - Quantity from transshipment production facility i to transshipment center j
Y = model_a.addVars(num_transship_facilities, num_transship_centers,lb=0, vtype=GRB.CONTINUOUS, name="Y")

# Zjk - Quantity from transshipment center j to refinement center k
Z = model_a.addVars(num_transship_centers, num_refinement_centers,lb=0, vtype=GRB.CONTINUOUS, name="Z")

In [9]:
#Define cost list
cost_values = costs_prod_ref['Cost'].to_numpy()

cost_prod_ref = cost_values.reshape((25, 5)).tolist()

cost_values = costs_prod_trans['Cost'].to_numpy()

cost_prod_trans = cost_values.reshape((15, 2)).tolist()

cost_values = costs_trans_ref['Cost'].to_numpy()

cost_trans_ref = cost_values.reshape((2, 5)).tolist()

In [10]:
demand_values = demand_data['Demand'].to_numpy()
demands = demand_values.reshape((1, 5)).tolist()
demands

[[1537, 1748, 1940, 1838, 1665]]

## Define objective function

In [11]:
prof_ref_objective = gb.quicksum(cost_prod_ref[i][j]*X[i,j] for i in range(25) for j in range(5))
prof_trans_objective = gb.quicksum(cost_prod_trans[i][j]*Y[i,j] for i in range(15) for j in range(2))
trans_ref_objective = gb.quicksum(cost_trans_ref[i][j]*Z[i,j] for i in range(2) for j in range(5))

In [12]:
# Objective function: Minimize total cost
model_a.setObjective(prof_ref_objective+prof_trans_objective+trans_ref_objective,
                   GRB.MINIMIZE)

## Define constraint

In [13]:
# Constraints
# Capacity constraints for direct production facilities
for i in range(num_direct_facilities):
    model_a.addConstr(gb.quicksum(X[i, j] for j in range(num_refinement_centers)) <= cap_direct[i])

# Capacity constraints for transshipment production facilities
for i in range(num_transship_facilities):
    model_a.addConstr(gb.quicksum(Y[i, j] for j in range(num_transship_centers)) <= cap_transship_facilities[i])
    
# Capacity constraints for transshipment production facilities
for i in range(num_transship_centers):
    model_a.addConstr(gb.quicksum(Z[i, j] for j in range(num_refinement_centers)) <= cap_transship_centers[i])

# Demand constraints at each refinement center
for j in range(num_refinement_centers):
    model_a.addConstr(gb.quicksum(X[i, j] for i in range(num_direct_facilities)) +
                    gb.quicksum(Z[k, j] for k in range(num_transship_centers)) == demands[0][j])
    
# Add the flow balance constrainits
model_a.addConstr(gb.quicksum(Y[i,0] for i in range(num_transship_facilities)) == gb.quicksum(Z[0,j] for j in range(num_refinement_centers)), name="Flow Balance 1")
model_a.addConstr(gb.quicksum(Y[i,1] for i in range(num_transship_facilities)) == gb.quicksum(Z[1,j] for j in range(num_refinement_centers)), name="Flow Balance 2")


<gurobi.Constr *Awaiting Model Update*>

## Optimize model

In [14]:
# Optimize the model
model_a.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49 rows, 165 columns and 340 nonzeros
Model fingerprint: 0x4b7d6154
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 49 rows, 165 columns, 340 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7230583e+04   1.238600e+04   0.000000e+00      0s
      36    2.4188585e+04   0.000000e+00   0.000000e+00      0s

Solved in 36 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.418858517e+04


In [15]:
# Print the decision variables
print(model_a.printAttr('X'))


    Variable            X 
-------------------------
      X[0,3]          462 
      X[1,1]          103 
      X[2,2]          460 
      X[4,3]           86 
      X[5,1]          217 
      X[7,4]          521 
      X[8,4]          548 
     X[10,4]          354 
     X[11,0]            7 
     X[11,2]          404 
     X[12,0]          104 
     X[13,4]          155 
     X[14,3]          285 
     X[15,0]          109 
     X[17,1]          351 
     X[17,4]           87 
     X[18,3]          501 
     X[20,2]          462 
     X[21,3]          504 
     X[22,1]          106 
     X[23,2]          132 
      Y[0,1]          374 
      Y[2,1]          395 
      Y[3,1]          245 
      Y[6,0]          152 
      Y[7,1]           24 
      Y[8,1]          415 
      Y[9,0]          503 
     Y[11,0]          297 
     Y[14,0]          365 
      Z[0,0]         1317 
      Z[1,1]          971 
      Z[1,2]          482 
None


## Question(a): After solving the linear program, what is the optimal transportation cost?

In [16]:
# Check if the solution is found
if model_a.status == GRB.Status.OPTIMAL:
    optimal_cost_a = model_a.objVal
    print(f"Optimal Transportation Cost_a: {optimal_cost_a}")
else:
    print("No optimal solution found")

Optimal Transportation Cost_a: 24188.585165447683


## Question(b): In the optimal solution, what proportion of canola oil is transshipped?

In [17]:
total_transshipped = sum(Y[i, j].X for i in range(num_transship_facilities) for j in range(num_transship_centers))

total_transport = total_transshipped + sum(X[i, j].X for i in range(num_direct_facilities) for j in range(num_refinement_centers))

proportion_transshipped = total_transshipped / total_transport

print('% of transshipped:',proportion_transshipped)


% of transshipped: 0.31736938588450964



 ## Question(c): The model does not currently limit that amount of canola oil that is transshipped. How would you modify the objective function to account for this? Formulate and solve this model.

In [18]:
total_transshipped

2770.0

In [19]:
# Create the optimization model
model_b = Model("limit the amount transshipped")

In [20]:
# Create decision variables and set the upperbond 1000 for Quantity from transshipment production facility i to transshipment center
# Xij - Quantity from direct production facility i to refinement center j
X_b = model_b.addVars(num_direct_facilities, num_refinement_centers,lb =0, vtype=GRB.CONTINUOUS, name="X_b")

# Yij - Quantity from transshipment production facility i to transshipment center j
Y_b = model_b.addVars(num_transship_facilities, num_transship_centers,lb=0, ub=1000, vtype=GRB.CONTINUOUS, name="Y_b")

# Zjk - Quantity from transshipment center j to refinement center k
Z_b = model_b.addVars(num_transship_centers, num_refinement_centers,lb=0, ub=1000, vtype=GRB.CONTINUOUS, name="Z_b")

In [21]:
# Objective function: Minimize total cost
prof_ref_objective_b = gb.quicksum(cost_prod_ref[i][j]*X_b[i,j] for i in range(25) for j in range(5))
prof_trans_objective_b = gb.quicksum(cost_prod_trans[i][j]*Y_b[i,j] for i in range(15) for j in range(2))
trans_ref_objective_b = gb.quicksum(cost_trans_ref[i][j]*Z_b[i,j] for i in range(2) for j in range(5))

model_b.setObjective(prof_ref_objective_b+prof_trans_objective_b+trans_ref_objective_b,
                   GRB.MINIMIZE)

In [22]:
cap_transship_facilities

array([374, 444, 395, 245, 378, 408, 435, 175, 415, 503, 184, 297, 450,
       169, 365], dtype=int64)

In [23]:
# Define Constraints
# Capacity constraints for direct production facilities
for i in range(num_direct_facilities):
    model_b.addConstr(gb.quicksum(X_b[i, j] for j in range(num_refinement_centers)) <= cap_direct[i])

# Capacity constraints for transshipment production facilities
for i in range(num_transship_facilities):
    model_b.addConstr(gb.quicksum(Y_b[i, j] for j in range(num_transship_centers)) <= cap_transship_facilities[i])

# Capacity constraints for transshipment production facilities
for i in range(num_transship_centers):
    model_b.addConstr(gb.quicksum(Z_b[i, j] for j in range(num_refinement_centers)) <= cap_transship_centers[i])

# Demand constraints at each refinement center
for j in range(num_refinement_centers):
    model_b.addConstr(gb.quicksum(X_b[i, j] for i in range(num_direct_facilities)) +
                    gb.quicksum(Z_b[k, j] for k in range(num_transship_centers)) == demands[0][j])
    
# Add the flow balance constrainits
model_b.addConstr(gb.quicksum(Y_b[i,0] for i in range(num_transship_facilities)) == gb.quicksum(Z_b[0,j] for j in range(num_refinement_centers)), name="Flow Balance 1")
model_b.addConstr(gb.quicksum(Y_b[i,1] for i in range(num_transship_facilities)) == gb.quicksum(Z_b[1,j] for j in range(num_refinement_centers)), name="Flow Balance 2")

<gurobi.Constr *Awaiting Model Update*>

In [24]:
model_b.optimize()
    
if model_b.status == GRB.Status.OPTIMAL:
    optimal_cost_b = model_b.objVal
    print(f"Optimal Transportation Cost_b: {optimal_cost_b}")
else:
    print("No optimal solution found")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49 rows, 165 columns and 340 nonzeros
Model fingerprint: 0xa40d3184
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [1e+03, 1e+03]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 49 rows, 165 columns, 340 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7230583e+04   3.113500e+03   0.000000e+00      0s
      37    2.4296202e+04   0.000000e+00   0.000000e+00      0s

Solved in 37 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.429620162e+04
Optimal Transportation Cost_b: 24296.201616368264


## Quesion(d) Instead of modifying the objective function, how would you modify the constraint set to reduce the proportion of canola oil that is transshipped? Formulate and solve this model.

In [25]:
print('% of transshipped:',proportion_transshipped)

% of transshipped: 0.31736938588450964


In [26]:
model_c = Model("contraint for proportion transshipped")

In [27]:
# Create decision variables
# Xij - Quantity from direct production facility i to refinement center j
X_c = model_c.addVars(num_direct_facilities, num_refinement_centers,lb =0, vtype=GRB.CONTINUOUS, name="X_c")

# Yij - Quantity from transshipment production facility i to transshipment center j
Y_c = model_c.addVars(num_transship_facilities, num_transship_centers,lb=0, vtype=GRB.CONTINUOUS, name="Y_c")

# Zjk - Quantity from transshipment center j to refinement center k
Z_c = model_c.addVars(num_transship_centers, num_refinement_centers,lb=0, vtype=GRB.CONTINUOUS, name="Z_c")

In [28]:
# Objective function: Minimize total cost
prof_ref_objective_c = gb.quicksum(cost_prod_ref[i][j]*X_c[i,j] for i in range(25) for j in range(5))
prof_trans_objective_c = gb.quicksum(cost_prod_trans[i][j]*Y_c[i,j] for i in range(15) for j in range(2))
trans_ref_objective_c = gb.quicksum(cost_trans_ref[i][j]*Z_c[i,j] for i in range(2) for j in range(5))

model_c.setObjective(prof_ref_objective_c+prof_trans_objective_c+trans_ref_objective_c,
                   GRB.MINIMIZE)

In [29]:
# add Constraint 0.35 to reduce the porportion of transshipment from 0.32 to 0.25
total_transshipped_c = sum(Y_c[i, j] for i in range(num_transship_facilities) for j in range(num_transship_centers))

total_transport_c = total_transshipped_c + sum(X_c[i, j] for i in range(num_direct_facilities) for j in range(num_refinement_centers))

model_c.addConstr(total_transshipped_c <= 0.15*total_transport_c)

<gurobi.Constr *Awaiting Model Update*>

In [30]:
# Capacity constraints for direct production facilities
for i in range(num_direct_facilities):
    model_c.addConstr(gb.quicksum(X_c[i, j] for j in range(num_refinement_centers)) <= cap_direct[i])

# Capacity constraints for transshipment production facilities
for i in range(num_transship_facilities):
    model_c.addConstr(gb.quicksum(Y_c[i, j] for j in range(num_transship_centers)) <= cap_transship_facilities[i])

# Capacity constraints for transshipment production facilities
for i in range(num_transship_centers):
    model_c.addConstr(gb.quicksum(Z_c[i, j] for j in range(num_refinement_centers)) <= cap_transship_centers[i])

# Demand constraints at each refinement center
for j in range(num_refinement_centers):
    model_c.addConstr(gb.quicksum(X_c[i, j] for i in range(num_direct_facilities)) +
                    gb.quicksum(Z_c[k, j] for k in range(num_transship_centers)) == demands[0][j])
    
# Add the flow balance constrainits
model_c.addConstr(gb.quicksum(Y_c[i,0] for i in range(num_transship_facilities)) == gb.quicksum(Z_c[0,j] for j in range(num_refinement_centers)), name="Flow Balance 1")
model_c.addConstr(gb.quicksum(Y_c[i,1] for i in range(num_transship_facilities)) == gb.quicksum(Z_c[1,j] for j in range(num_refinement_centers)), name="Flow Balance 2")

<gurobi.Constr *Awaiting Model Update*>

In [31]:
# optimize the model    
model_c.optimize()

print('\n')
# Check if the solution is found
if model_c.status == GRB.Status.OPTIMAL:
    optimal_cost_c = model_c.objVal
    print(f"Optimal Transportation Cost_d: {optimal_cost_c}")
else:
    print("No optimal solution found")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50 rows, 165 columns and 495 nonzeros
Model fingerprint: 0xac56c5f3
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 50 rows, 165 columns, 495 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7230583e+04   1.238600e+04   0.000000e+00      0s
      39    2.6259090e+04   0.000000e+00   0.000000e+00      0s

Solved in 39 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.625908978e+04


Optimal Transportation Cost_d: 26259.089782824856


## Question (e) Which of the two modeling approaches would you recommend the company take to determine a transportation plan that reduces the amount of canola oil that is transshipped?

The approach in question(d) would be recommended: modify the constraint set to reduce the proportion of canola oil that is transshipped as there is more flexibility of setting up the constraints by amount or percentage

## Question(f): re-shoring is the practice of transferring overseas business operations closer to the home country. Given its prevalence in today’s economy, how would you alter the original model to favor producers closer to North America? Formulate and solve this model.


In [32]:
# create model
model_d = Model("Re-shoring problem")

In [33]:
# Create decision variables
# Xij - Quantity from direct production facility i to refinement center j
X_d = model_d.addVars(num_direct_facilities, num_refinement_centers,lb =0, vtype=GRB.CONTINUOUS, name="X")

# Yij - Quantity from transshipment production facility i to transshipment center j
Y_d = model_d.addVars(num_transship_facilities, num_transship_centers,lb=0, vtype=GRB.CONTINUOUS, name="Y")

# Zjk - Quantity from transshipment center j to refinement center k
Z_d = model_d.addVars(num_transship_centers, num_refinement_centers,lb=0, vtype=GRB.CONTINUOUS, name="Z")

In [34]:
# define Objective function: Minimize total cost
prof_ref_objective_d = gb.quicksum(cost_prod_ref[i][j]*X_d[i,j] for i in range(25) for j in range(5))
prof_trans_objective_d = gb.quicksum(cost_prod_trans[i][j]*Y_d[i,j] for i in range(15) for j in range(2))
trans_ref_objective_d = gb.quicksum(cost_trans_ref[i][j]*Z_d[i,j] for i in range(2) for j in range(5))

model_d.setObjective(prof_ref_objective_d+prof_trans_objective_d+trans_ref_objective_d,
                   GRB.MINIMIZE)

In [35]:
north_america_direct_facilities = range(15)
nonnorth_america__direct_facilities = range(15, 25)

In [36]:
# Capacity constraints for North America direct production facilities
for i in north_america_direct_facilities:
    model_d.addConstr(gb.quicksum(X_d[i, j] for j in range(num_refinement_centers)) == cap_direct[i])
    
# Capacity constraints for Non-North America direct production facilities   
for i in nonnorth_america__direct_facilities:
    model_d.addConstr(gb.quicksum(X_d[i, j] for j in range(num_refinement_centers)) <= cap_direct[i-15])
    
# Capacity constraints for transshipment production facilities
for i in range(num_transship_facilities):
    model_d.addConstr(gb.quicksum(Y_d[i, j] for j in range(num_transship_centers)) <= cap_transship_facilities[i])

# Capacity constraints for transshipment production facilities
for i in range(num_transship_centers):
    model_d.addConstr(gb.quicksum(Z_d[i, j] for j in range(num_refinement_centers)) <= cap_transship_centers[i])

# Demand constraints at each refinement center
for j in range(num_refinement_centers):
    model_d.addConstr(gb.quicksum(X_d[i, j] for i in range(num_direct_facilities)) +
                    gb.quicksum(Z_d[k, j] for k in range(num_transship_centers)) == demands[0][j])
    
# Add the flow balance constrainits
model_d.addConstr(gb.quicksum(Y_d[i,0] for i in range(num_transship_facilities)) == gb.quicksum(Z_d[0,j] for j in range(num_refinement_centers)), name="Flow Balance 1")
model_d.addConstr(gb.quicksum(Y_d[i,1] for i in range(num_transship_facilities)) == gb.quicksum(Z_d[1,j] for j in range(num_refinement_centers)), name="Flow Balance 2")

<gurobi.Constr *Awaiting Model Update*>

In [37]:
model_d.optimize()

if model_d.status == GRB.Status.OPTIMAL:
    optimal_cost_d = model_d.objVal
    print(f"Optimal Transportation Cost_f: {optimal_cost_d}")
else:
    print("No optimal solution found")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49 rows, 165 columns and 340 nonzeros
Model fingerprint: 0x9c13b14a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 49 rows, 165 columns, 340 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4449141e+04   4.153000e+03   0.000000e+00      0s
      27    2.4946141e+04   0.000000e+00   0.000000e+00      0s

Solved in 27 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.494614078e+04
Optimal Transportation Cost_f: 24946.14078171339


## Question (g) Do you expect the optimal solution to the re-shoring model to be similar to the optimal solution of the model that attempts to reduce transshipment? Why or why not?

No. the constraints are different

In [38]:
# create a dictionalyr
results = {
    "Model A": optimal_cost_a,
    "Model B": optimal_cost_b,
    "Model C": optimal_cost_c,
    "Model D": optimal_cost_d
}

# convert to dataframe
results_df = pd.DataFrame(list(results.items()), columns=["Model", "Optimal Cost"])

sorted_results_df = results_df.sort_values(by="Optimal Cost")

print(sorted_results_df)

best_model = sorted_results_df.iloc[0]
print(f"The best solution is provided by {best_model['Model']} with a cost of {best_model['Optimal Cost']}")

     Model  Optimal Cost
0  Model A  24188.585165
1  Model B  24296.201616
3  Model D  24946.140782
2  Model C  26259.089783
The best solution is provided by Model A with a cost of 24188.585165447683
